In [1]:
from transformers import pipeline
classifier = pipeline("text-classification", model="distilbert-base-uncased-finetuned-sst-2-english",
                      truncation = True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [2]:
import pandas as pd

teslamotors = pd.read_csv("teslamotors.csv")
electric_vehicles = pd.read_csv("electric_vehicles.csv")
fuckcars = pd.read_csv("fuckcars.csv")
teslamodel3 = pd.read_csv("teslamdel3.csv")
rivian = pd.read_csv("rivian.csv")

In [12]:
def sentiment_analysis(text_series):
  text_series = text_series.fillna("").astype(str)
  results = text_series.apply(lambda x: classifier(x)[0])
  return results.apply(lambda result: (result['label'], result['score'])).tolist()

In [13]:
def sentiment_df(df):
    title_results = sentiment_analysis(df['cleaned_title'])
    text_results = sentiment_analysis(df['cleaned_text'])

    # Create new columns for sentiment labels and scores
    df[['sentiment_title', 'sentiment_title_score']] = pd.DataFrame(title_results, index=df.index)
    df[['sentiment_text', 'sentiment_text_score']] = pd.DataFrame(text_results, index=df.index)

    return df

In [14]:
teslamotors = sentiment_df(teslamotors)
fuckcars = sentiment_df(fuckcars)
teslamodel3 = sentiment_df(teslamodel3)
rivian = sentiment_df(rivian)
electric_vehicles = sentiment_df(electric_vehicles)

In [15]:
import numpy as np
def sentiment_match(df):
  df['sentiment_text'] = np.where(df['cleaned_text'].isna(),
                                  df['sentiment_title'],
                                  df['sentiment_text'])

  df['sentiment_title'] = np.where(df['cleaned_title'].isna(),
                                   df['sentiment_text'],
                                   df['sentiment_title'])
  return(df)

In [17]:
teslamotors = sentiment_match(teslamotors)
fuckcars = sentiment_match(fuckcars)
rivian = sentiment_match(rivian)
teslamodel3 = sentiment_match(teslamodel3)
electric_vehicles = sentiment_match(electric_vehicles)

In [55]:
def overall_sentiment(df):
    # Identify sarcasm
    df['sarcasm'] = df.apply(lambda row: row['sentiment_title'] != row['sentiment_text'], axis=1)

    # Filter out rows with sarcasm
    df_filtered = df[df['sarcasm'] == False].copy()

    # Calculate overall sentiment score as the max of title and text sentiment scores
    df_filtered['overall_sentiment_score_post'] = df_filtered.apply(
        lambda row: max(row['sentiment_title_score'], row['sentiment_text_score'])*
        (-1 if row['sentiment_title'] == 'NEGATIVE' else 1),
        axis=1
    )

    df_filtered['overall_sentiment_post'] = df_filtered['sentiment_title']
    drop_list = ['sarcasm', 'sentiment_title', 'sentiment_text', 'sentiment_title_score', 'sentiment_text_score']
    df_filtered = df_filtered.drop(drop_list, axis = 1)

    return df_filtered

In [56]:
teslamotors = overall_sentiment(teslamotors)
fuckcars = overall_sentiment(fuckcars)
rivian = overall_sentiment(rivian)
teslamodel3 = overall_sentiment(teslamodel3)
electric_vehicles = overall_sentiment(electric_vehicles)

In [58]:
teslamotors.to_csv('teslamotors_sentiment.csv',
                   index=False,
                   sep=',',
                   header=True,
                   na_rep='NA')
fuckcars.to_csv('fuckcars_sentiment.csv',
                index=False,
                sep=',',
                header=True,
                na_rep='NA')
teslamodel3.to_csv('teslamodel3_sentiment.csv',
                   index=False,
                   sep=',',
                   header=True,
                   na_rep='NA')
rivian.to_csv('rivian_sentiment.csv',
              index=False,
              sep=',',
              header=True,
              na_rep='NA')
electric_vehicles.to_csv('electric_vehicles_sentiment.csv',
                         index=False,
                         sep=',',
                         header=True,
                         na_rep='NA')

In [57]:
teslamotors.columns.tolist()

['date_utc',
 'url',
 'comments',
 'matched_keyword',
 'cleaned_title',
 'cleaned_text',
 'overall_sentiment_score_post',
 'overall_sentiment_post']

In [61]:
print(np.sum(teslamotors['comments']))
print(np.sum(fuckcars['comments']))
print(np.sum(electric_vehicles['comments']))
print(np.sum(teslamodel3['comments']))
print(np.sum(rivian['comments']))

139498
85937
186358
54672
29739


In [83]:
print(comments_df)

Empty DataFrame
Columns: []
Index: []
